In [1]:
import tensorflow as tf
import json
import matplotlib.pyplot as plt
import numpy as np
import cv2
import copy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import load_img
from keras.preprocessing.image import save_img

from PIL import Image, ImageDraw, ImageOps , ImageFilter
from skimage.util import random_noise
from numpy import asarray

Using TensorFlow backend.


In [2]:
# soubor s anotacemi
annotation_path = '../CoralClef2020/annotations/annotation.json'
# cesta kam ma byt ulozen soubor augmentovanymi anotacemi
annotation_aug_path = '../CoralClef2020/annotations/'
# cesta k datasetu
dataset_path = "../CoralClef2020/training_set_2020/"
# cesta kam budou augmentovana data ulozena
augmented_path = "../CoralClef2020/training_set_2020_augmented/"

In [3]:
with open(annotation_path) as json_file:
    annotations = json.load(json_file)

In [368]:
data = copy.deepcopy(annotations)

progbar = tf.keras.utils.Progbar(len(annotations['images']))


for img_data in annotations['images']:
    img_id = img_data['id']
    img_name = img_data['file_name']
    f = dataset_path + img_name
    original_image = Image.open(f)
    
    progbar.update(img_id)
    # pocet obrazku ktere maji byt generovany z puvodniho
    n = np.random.randint(4)+1
    # pocet augmentaci ktere budou aplikovany na obrazek
    m = 2
    for i in range(n):
        
        augment = np.random.randint(m, size=4)
        while(sum(augment) == 0):
            augment = np.random.randint(m, size=4)
        
        new_img = original_image.copy()
        new_name = img_name.split('.')[0] + "_" + str(i) + ".jpg"
        img_id = img_data['id']
        
        new_ann = {'images' : [], 'annotations' : []}
        new_ann['images'].append(copy.deepcopy(img_data))
        for i in annotations['annotations']:
             if i['image_id'] == img_id:
                    new_ann['annotations'].append(copy.deepcopy(i))
                    
        if augment[0] == 1:
            new_img, new_ann = crop(new_img, img_id, new_ann, data)
            img_id = new_ann['images'][0]['id']
        if augment[1] == 1:
            new_img, new_ann = flip_h(new_img, img_id, new_ann, data)
            img_id = new_ann['images'][0]['id']
        if augment[2] == 1:
            new_img, new_ann = blur(new_img, img_id, new_ann, data)
            img_id = new_ann['images'][0]['id']
        if augment[3] == 1:
            new_img, new_ann = color_change(new_img, img_id, new_ann, data)
            img_id = new_ann['images'][0]['id']
        
        new_ann['images'][0]['file_name'] = new_name
        data['images'].append(new_ann['images'][0])
        for i in new_ann['annotations']:
            data['annotations'].append(i)
        new_img.save(augmented_path + new_name) 
    original_image.save(augmented_path + img_name)

    
with open(annotation_aug_path + 'annotation_augment.json', 'w') as file:
    json.dump(data, file)

439/440 [============================>.] - ETA: 13s

In [355]:
def last_id(d):
    max_i = 0
    for i in d['images']:
        if int(i['id']) > max_i:
            max_i = int(i['id'])
    max_a = 0        
    for i in d['annotations']:
        if int(i['id']) > max_a:
            max_a = int(i['id'])
    return (max_i, max_a)

def visualize(original, augmented):
    fig = plt.figure()
    plt.subplot(1,2,1)
    plt.title('Original image')
    plt.imshow(original)

    plt.subplot(1,2,2)
    plt.title('Augmented image')
    plt.imshow(augmented)

# BLUR

In [356]:
def blur(o_img, img_id, lastAnotation, ann_data):

    lastid_img, lastid_ano = last_id(ann_data)
    newAnotation = {'images': [{
        'file_name': '',
        'url': '',
        'height': str(o_img.size[1]),
        'width': str(o_img.size[0]),
        'id': int(lastid_img+1)}], 
     'annotations': []}

    image_noise = o_img.filter(ImageFilter.GaussianBlur(radius=8))

    k = 1
    for i in lastAnotation['annotations']:
        if i['image_id'] == img_id:
            
            a = copy.deepcopy(i)
            a['image_id'] = lastid_img + 1
            a['id'] = lastid_ano + k
            k = k + 1
            newAnotation['annotations'].append(a)

    return image_noise, newAnotation

# FLIP HPROZONTAL

In [357]:
def flip_h(o_img, img_id, lastAnotation, ann_data):

    lastid_img, lastid_ano = last_id(ann_data)
    newAnotation = {'images': [{
        'file_name': '',
        'url': '',
        'height': str(o_img.size[1]),
        'width': str(o_img.size[0]),
        'id': int(lastid_img+1)}], 
     'annotations': []}
    

    image_fliph = ImageOps.mirror(o_img)
    
    k = 1
    for i in lastAnotation['annotations']:
        if i['image_id'] == img_id:
            bb = i['bbox']
            w = image_fliph.size[0]
            
            a = copy.deepcopy(i)
            a['image_id'] = lastid_img + 1
            a['id'] = lastid_ano + k
            k = k + 1
            a['bbox'] = [str(w-int(bb[0])), bb[1], str(w-int(bb[2])), bb[3]]

            for s in range(0, len(a['segmentation']), 2):
                sx = w - int(a['segmentation'][s])
                sy = int(a['segmentation'][s+1])
                if(sx<0):
                    sx = 0

                a['segmentation'][s] = str(sx)
                a['segmentation'][s+1] = str(sy)

            newAnotation['annotations'].append(a)

    return image_fliph, newAnotation

# CROP

In [359]:
def crop(o_i, image_id, lastAnotation, ann_data):
    size = o_i.size
    w = size[0]
    h = size[1]
    r = np.random.uniform()

    if(r < 0.2):
        x1 = 0
        y1 = 0
    elif(r > 0.2 and r < 0.4):
        x1 = int(w/2)
        y1 = 0
    elif(r > 0.4 and r < 0.6):
        x1 = 0
        y1 = int(h/2)
    elif(r > 0.6 and r < 0.8):
        x1 = int(w/2)
        y1 = int(h/2)
    else:
        x1 = int(w/4)
        y1 = int(h/4)

    x2 = int(x1 + w/2)
    y2 = int(y1 + h/2)
    

    lastid_img, lastid_ano = last_id(ann_data)
    
    newAnotation = {'images': [{
        'file_name': '',
        'url': '',
        'height': str(y2),
        'width': str(x2),
        'id': int(lastid_img+1)}], 
     'annotations': []}
    
    area = (x1, y1, x2, y2)
    cropped_img = o_i.crop(area)
    
    k = 1
    for i in lastAnotation['annotations']:
        if i['image_id'] == image_id:
            bb = i['bbox']
            s1 = int(int(bb[0])-x1)
            s2 = int(int(bb[3])-y1)
            e1 = int(int(bb[2])-x1)
            e2 = int(int(bb[1])-y1)
            
            if s1 < 0:
                s1 = 0
            if s1 > x2-x1:
                s1 = x2-x1
                
            if s2 < 0:
                s2 = 0
            if s2 > y2-y1:
                s2 = y2-y1   
                
            if(e1 < 0):
                e1 = 0   
            if e1 > x2-x1:
                e1 = x2-x1
                
            if(e2 < 0):
                e2 = 0
            if e2 > y2-y1:
                e2 = y2-y1
            
            
            image2 = cropped_img.copy()
            if(s1 != e1 and s2 != e2):
                a = copy.deepcopy(i)
                a['image_id'] = lastid_img + 1
                a['id'] = lastid_ano + k
                k = k + 1
                a['bbox'] = [str(s1), str(e2), str(e1), str(s2)]
                for s in range(0, len(a['segmentation']), 2):
                    sx = int(a['segmentation'][s])-x1
                    sy = int(a['segmentation'][s+1])-y1
                    if sx < 0:
                        sx = 0
                    if sy < 0:
                        sy = 0
                    a['segmentation'][s] = str(sx)
                    a['segmentation'][s+1] = str(sy)
                    

                newAnotation['annotations'].append(a)
                color = (255, 0, 0)

    return cropped_img, newAnotation

# COLOR CHANGE

In [360]:
def color_change(o_img, img_id, lastAnotation, ann_data):
    
    lastid_img, lastid_ano = last_id(ann_data)
    newAnotation = {'images': [{
        'file_name': '',
        'url': '',
        'height': str(o_img.size[1]),
        'width': str(o_img.size[0]),
        'id': int(lastid_img+1)}], 
     'annotations': []}

    
    image = o_img.copy()
    pixels = image.load()
    (r, g, b) = avg_colors_img(o_img)
    for i in range(0, int(o_img.size[0]),1): 
            for j in range(0, int(o_img.size[1]),1):
                rr = pixels[i,j][0]
                gg = pixels[i,j][1]
                bb = pixels[i,j][2]

                if(g/b > 1.5):
                    bb = bb+50
                    gg = gg-10
                else:
                    gg = gg+10
                    bb = bb-50 

                pixels[i,j] = (int(rr), int(gg), int(bb))
    k = 1
    for i in lastAnotation['annotations']:
        if i['image_id'] == img_id:

            a = copy.deepcopy(i)
            a['image_id'] = lastid_img + 1
            a['id'] = lastid_ano + k
            k = k + 1
            newAnotation['annotations'].append(a)
            
    return image, newAnotation

In [361]:
def avg_colors_img(o_i):
    r_avg = 0
    g_avg = 0
    b_avg = 0
    
    pixels = original_image.load()
    
    k = 0
    for i in range(0,o_i.size[0],2):
        for j in range(0,o_i.size[1],2):
            k += 1
            #r_avg += pixels[i,j][0]
            g_avg += pixels[i,j][1]
            b_avg += pixels[i,j][2]

    #r_avg /= k
    g_avg /= k
    b_avg /= k
    return (r_avg, g_avg, b_avg)


def avg_colors_all():
    r = []
    g = []
    b = []


    progbar = tf.keras.utils.Progbar(439)
    for i in range(0,439):
        f = dataset_path + data['images'][i]['file_name']
        original_image = Image.open(f)
        (rr, gg, bb) = avg_colors_img(original_image)
        r.append(rr)
        g.append(gg)
        b.append(bb)
        progbar.update(i)
    
    return(r, g, b)